In [1]:
import requests
import bs4
import os
import pandas as pd

In [2]:
import datetime

now = datetime.datetime.now()


Today_time = now.strftime("%H:%M")

Today_date = now.strftime("%Y-%m-%d")

In [3]:
def record_date_time(z):
    x=str.split(z,',')
    date =x[0]
    time= x[1] +':00'

    return date,time

def record_URL(x):
    y='https://timesofindia.indiatimes.com'
    url = y+x 
    return url

In [4]:
link ='https://timesofindia.indiatimes.com/'
res = requests.get(link)
soup =bs4.BeautifulSoup(res.text,'lxml')

In [5]:
List = soup.find('li',{'class':'nav-Life & Style'}).find('div',{'class':'left-column simple-list'}).findAll('li')

In [6]:
records = []
for link in List[1:-2]:
    url = record_URL(link.find('a')['href'])
    category = link.find('a').text.lower()
    if(category=='recipes' or category=='travel'):
        continue
    res = requests.get(url)
    soup =bs4.BeautifulSoup(res.text,'lxml')

    main = soup.find('div',{'class':'nent_pink_header clearfix'}).findAll('li',{'class':'l3hovermenu'})
    if main:
        for content in main:
            try:
                sub_url = content.find('a')['href']
                sub_category = content.find('a').text.lower()
                sub_res = requests.get(sub_url)
                sub_soup =bs4.BeautifulSoup(sub_res.text,'lxml').find('div',{'id':'mainlisting'}).findAll('div',{'class':'md_news_box'})

                for record in sub_soup:
                    try:
                        Heading = sub_category.capitalize()  +': '+record.text
                        URL = record_URL(record.find('a')['href'])
                        Date,Time = record_date_time(record.findAll('span')[-1]['dateval'])
                        Category = category
                        Source = 'Time of India'
                    except:
                        continue
                    records.append((Source,Heading,Category,Date,Time,URL))    
            except:
                continue
    else:
        try:
            
            Heading = sub_category.capitalize()  +': '+record.text
            URL = record_URL(record.find('a')['href'])
            Date,Time = record_date_time(record.findAll('span')[-1]['dateval'])
            Category = category
            Source = 'Times of India'
        except:
            continue
        records.append((Source,Heading,Category,Date,Time,URL))    
        
   

In [7]:
sub_category

'videos'

In [8]:
import pandas as pd
df=pd.DataFrame(records,columns=['Source','Heading','Category','Date','Time','URL'])
df['Date'] = pd.to_datetime(df['Date'])

In [9]:
df

,Source,Heading,Category,Date,Time,URL
0,Time of India,Love & sex: Signs that overthinking is killing...,relationships,2019-06-21,08:00:00,https://timesofindia.indiatimes.com/life-style...
1,Time of India,Love & sex: How yoga can help strengthen your ...,relationships,2019-06-21,14:04:00,https://timesofindia.indiatimes.com/life-style...
2,Time of India,Love & sex: He made me cry every night: Here's...,relationships,2019-06-21,10:30:00,https://timesofindia.indiatimes.com/life-style...
3,Time of India,Love & sex: Things we should stop doing at Ind...,relationships,2019-06-20,08:00:00,https://timesofindia.indiatimes.com/life-style...
4,Time of India,Love & sex: Harsh truths about marriage that n...,relationships,2019-06-19,08:00:00,https://timesofindia.indiatimes.com/life-style...
5,Time of India,Love & sex: The tricky task of deciding the we...,relationships,2019-06-18,11:51:00,https://timesofindia.indiatimes.com/life-style...
6,Time of India,Love & sex: Priyanka Chopra’s hanks to her in-...,relationships,2019-06-17,17:26:00,https://timesofindia.indiatimes.com/life-style...
7,Time of India,Love & sex: How a person tilts his head can ma...,relationships,2019-06-16,17:02:00,https://timesofindia.indiatimes.com/life-style...
8,Time of India,Love & sex: Science suggests ‘love at first si...,relationships,2019-06-19,10:30:00,https://timesofindia.indiatimes.com/life-style...
9,Time of India,Love & sex: Is your love life full of struggle...,relationships,2019-06-15,09:00:00,https://timesofindia.indiatimes.com/life-style...


In [10]:
outname ='TOI_lifestyle-'+str(Today_date)+'.csv'

root = 'CSV/'
if not os.path.exists(root):
    os.mkdir(root)
date_today= Today_date +'/'
outdir=root+ date_today[:-1]
if not os.path.exists(outdir):
    os.mkdir(outdir)
fullname = os.path.join(outdir, outname)

In [11]:
df.to_csv(fullname, index=False, encoding='utf-8')